In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

# Setting maximum number of rows and columns to display in the notebook
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)

## Exploring the Airport Codes data

In [2]:
airports = pd.read_csv('Airport_Codes.csv')

#### Size of the data

In [3]:
airports.shape

(55369, 8)

We have data corresponding to 55369 airports assuming each row corresponds to a unique airport

#### Sample data

In [4]:
airports.head()

,TYPE,NAME,ELEVATION_FT,CONTINENT,ISO_COUNTRY,MUNICIPALITY,IATA_CODE,COORDINATES
0,heliport,Total Rf Heliport,11.0,NaN,US,Bensalem,NaN,"-74.93360137939453, 40.07080078125"
1,small_airport,Aero B Ranch Airport,3435.0,NaN,US,Leoti,NaN,"-101.473911, 38.704022"
2,small_airport,Lowell Field,450.0,NaN,US,Anchor Point,NaN,"-151.695999146, 59.94919968"
3,small_airport,Epps Airpark,820.0,NaN,US,Harvest,NaN,"-86.77030181884766, 34.86479949951172"
4,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,Newport,NaN,"-91.254898, 35.6087"


#### Checking the data types of all columns

In [5]:
airports.dtypes

TYPE             object
NAME             object
ELEVATION_FT    float64
CONTINENT        object
ISO_COUNTRY      object
MUNICIPALITY     object
IATA_CODE        object
COORDINATES      object
dtype: object

We can see that the data is not clean since there are missing values for the continent to which the airport belongs and the IATA code for the corresponding airport

#### Checking the null counts for all columns

In [6]:
airports.isna().sum()

TYPE                0
NAME                0
ELEVATION_FT     7015
CONTINENT       27843
ISO_COUNTRY       247
MUNICIPALITY     5706
IATA_CODE       46187
COORDINATES         0
dtype: int64

We are primarily interested in only medium and large size airports based in US. So we will first filter the data for these criteria and then carry on with our analysis.  
  
Basic check on the data shows that there are no null values in the TYPE column and hence we do not need to do any pre processing/imputation on the TYPE column before filtering.  
  
**Data Limitation:** We can observe that there are **247 NULL** values in **ISO_COUNTRY** column. This is a potential issue since we want to **limit our analysis on for domestic US airports**. We can observe that there is **NO NULL** value in the **COORDINATES** column. We can potentially use this data to obtain the missing datapoints for ISO_COUNTRY columns.  
  
We will first filter the data by TYPE for 'Medium' and 'Large' size airports to see if the issue still exists

### Filtering for Medium and Large airports

In [7]:
airports['TYPE'].value_counts()

small_airport     34120
heliport          11384
medium_airport     4532
closed             3680
seaplane_base      1016
large_airport       614
balloonport          23
Name: TYPE, dtype: int64

There are **4532 Medium** sized and **614 Large** sized airports

In [8]:
filter_condition = ['large_airport','medium_airport']
airport_subset = airports[airports['TYPE'].isin(filter_condition)]

#### Examining the NULL value counts again for the subset of the data

In [9]:
airport_subset.isna().sum()

TYPE               0
NAME               0
ELEVATION_FT     204
CONTINENT       1448
ISO_COUNTRY       12
MUNICIPALITY     535
IATA_CODE        687
COORDINATES        0
dtype: int64

**Percentage counts**

In [10]:
round(airport_subset.isna().sum()*100/airport_subset.isna().sum().sum(),2)

TYPE             0.00
NAME             0.00
ELEVATION_FT     7.07
CONTINENT       50.17
ISO_COUNTRY      0.42
MUNICIPALITY    18.54
IATA_CODE       23.80
COORDINATES      0.00
dtype: float64

We can see that the NULL value count dropped from 247 to 12 that corresponds to 0.42% of the data so we will ignore these rows and further filter by ISO_COUNTRY = US

### Filtering for US based airports

In [11]:
airport_subset['ISO_COUNTRY'].value_counts()

US    858
CA    338
CN    288
RU    272
AU    191
FR    144
BR    128
IN    118
JP    111
GB    101
DE     81
IT     76
AR     70
MX     66
ZA     63
IR     62
CO     60
TR     58
ID     55
PH     55
ES     52
SE     50
VE     49
NO     46
TH     42
PK     40
NZ     38
DZ     37
SA     36
CL     33
UA     32
PT     32
IQ     31
FI     31
PE     31
BS     29
PL     29
EG     28
GR     28
MY     28
PF     27
KZ     26
CU     26
NG     25
MG     23
KR     23
MM     22
TW     21
VN     21
CD     21
AO     21
RO     21
PG     21
CR     17
KP     17
DK     17
KE     17
MA     16
EC     16
NL     15
MZ     15
BO     15
CH     14
LY     14
ET     13
TZ     12
CZ     12
TN     12
AZ     12
BE     11
BZ     11
NC     11
BY     11
GE     11
IE     11
YE     11
GA     11
BG     11
AE     11
IL     11
DO     10
SY     10
HU     10
MN     10
CM     10
LK     10
SN     10
IS      9
BD      9
ZW      9
ZM      8
SK      8
SD      8
HN      8
AF      8
HR      8
PA      8
AT      8
GT      7
PR      7


In [12]:
airport_subset[airport_subset['ISO_COUNTRY'].isin(['US','UM'])]['ISO_COUNTRY'].value_counts()

US    858
UM      2
Name: ISO_COUNTRY, dtype: int64

The ISO_COUNTRY code UM corresponds to US Minor Outlying Islands. There are only 2 such airports and hence we are not considering them for further analysis

In [13]:
US_airport_subset = airport_subset[airport_subset['ISO_COUNTRY'] == 'US']

#### Size of the subset

In [14]:
US_airport_subset.shape

(858, 8)

#### Checking for NULL IATA codes

In [15]:
US_airport_subset['IATA_CODE'].isna().sum()

37

**We require IATA codes for joining the data with the ticket and the flight dataset. Hence we will drop the rows with NULL IATA codes**

In [16]:
US_airport_subset = US_airport_subset[~US_airport_subset['IATA_CODE'].isna()]

In [17]:
US_airport_subset.shape

(821, 8)

**We will use this subset of 821 US airports for further analysis.**

## Exploring the Tickets data

In [18]:
tickets = pd.read_csv('Tickets.csv')
tickets.shape

(1167285, 12)

In [19]:
tickets.head()

,ITIN_ID,YEAR,QUARTER,ORIGIN,ORIGIN_COUNTRY,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ROUNDTRIP,REPORTING_CARRIER,PASSENGERS,ITIN_FARE,DESTINATION
0,201912723049,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,736.0,DAB
1,201912723085,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,570.0,COS
2,201912723491,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,564.0,MCO
3,201912723428,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,345.0,LGA
4,201912723509,2019,1,ABI,US,TX,Texas,0.0,MQ,1.0,309.0,MGM


In [20]:
tickets.dtypes

ITIN_ID                int64
YEAR                   int64
QUARTER                int64
ORIGIN                object
ORIGIN_COUNTRY        object
ORIGIN_STATE_ABR      object
ORIGIN_STATE_NM       object
ROUNDTRIP            float64
REPORTING_CARRIER     object
PASSENGERS           float64
ITIN_FARE             object
DESTINATION           object
dtype: object

In [21]:
tickets.isna().sum()

ITIN_ID                 0
YEAR                    0
QUARTER                 0
ORIGIN                  0
ORIGIN_COUNTRY          0
ORIGIN_STATE_ABR        0
ORIGIN_STATE_NM         0
ROUNDTRIP               0
REPORTING_CARRIER       0
PASSENGERS           1977
ITIN_FARE             960
DESTINATION             0
dtype: int64

In [22]:
tickets['ROUNDTRIP'].value_counts()

1.0    708600
0.0    458685
Name: ROUNDTRIP, dtype: int64

In [23]:
tickets = tickets[tickets['ROUNDTRIP'] == 1.0]

In [24]:
tickets = pd.merge(tickets,
                   US_airport_subset['IATA_CODE'],
                   left_on = 'ORIGIN',
                   right_on = 'IATA_CODE',
                   how = 'inner')

tickets.drop(['IATA_CODE'], axis = 1, inplace = True)

tickets = pd.merge(tickets,
                   US_airport_subset['IATA_CODE'],
                   left_on = 'DESTINATION',
                   right_on = 'IATA_CODE',
                   how = 'inner')
tickets.drop(['IATA_CODE'], axis = 1, inplace = True)

In [25]:
tickets.isna().sum()

ITIN_ID                 0
YEAR                    0
QUARTER                 0
ORIGIN                  0
ORIGIN_COUNTRY          0
ORIGIN_STATE_ABR        0
ORIGIN_STATE_NM         0
ROUNDTRIP               0
REPORTING_CARRIER       0
PASSENGERS           1162
ITIN_FARE             545
DESTINATION             0
dtype: int64

In [26]:
tickets.describe()

,ITIN_ID,YEAR,QUARTER,ROUNDTRIP,PASSENGERS
count,6.842080e+05,684208.0,684208.0,684208.0,683046.000000
mean,1.729789e+11,2019.0,1.0,1.0,1.938763
std,6.713302e+10,0.0,0.0,0.0,5.090569
min,2.019119e+06,2019.0,1.0,1.0,1.000000
25%,2.019116e+11,2019.0,1.0,1.0,1.000000
50%,2.019128e+11,2019.0,1.0,1.0,1.000000
75%,2.019140e+11,2019.0,1.0,1.0,1.000000
max,2.019153e+11,2019.0,1.0,1.0,681.000000


In [27]:
def clean_value(value):
    
    value = re.sub(r'[^0-9.]+','', str(value))
    
    try:
        value = value.split('.')[0]
    except:
        value = np.nan
    
    return value

In [28]:
tickets['ITIN_FARE'] = tickets['ITIN_FARE'].apply(lambda x: clean_value(x))
tickets['ITIN_FARE'] = tickets['ITIN_FARE'].replace('',np.nan).astype('float64')

In [29]:
tickets['ITIN_FARE'].isna().sum()

545

In [30]:
tickets['TRIP'] = tickets['ORIGIN'] + '-' + tickets['DESTINATION']

In [31]:
tickets['TRIP'].nunique()

37744

In [32]:
avg_ticket_price = tickets.groupby('TRIP')['ITIN_FARE'].mean().reset_index().rename(columns={'ITIN_FARE':'AVG_FARE'})
avg_ticket_price

,TRIP,AVG_FARE
0,ABE-ABQ,534.000000
1,ABE-AGS,299.000000
2,ABE-AMA,654.000000
3,ABE-ASE,742.000000
4,ABE-ATL,505.038462
...,...,...
37739,YUM-SMF,276.500000
37740,YUM-SRQ,696.000000
37741,YUM-TOL,335.000000
37742,YUM-TPA,544.333333


In [33]:
avg_ticket_price.isna().sum()

TRIP        0
AVG_FARE    8
dtype: int64

In [34]:
avg_ticket_price[avg_ticket_price['AVG_FARE'].isna()]

,TRIP,AVG_FARE
5148,BTR-MLI,NaN
5316,BTV-SWF,NaN
8224,CPR-EUG,NaN
11204,ECP-FWA,NaN
14047,GEG-EWN,NaN
17371,IDA-EKO,NaN
27584,PHL-SJT,NaN
32983,SFO-ISP,NaN


In [35]:
tickets[tickets['TRIP'].isin(avg_ticket_price[avg_ticket_price['AVG_FARE'].isna()]['TRIP'])]

,ITIN_ID,YEAR,QUARTER,ORIGIN,ORIGIN_COUNTRY,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ROUNDTRIP,REPORTING_CARRIER,PASSENGERS,ITIN_FARE,DESTINATION,TRIP
597004,201911821379,2019,1,ECP,US,FL,Florida,1.0,DL,1.0,NaN,FWA,ECP-FWA
640916,201912734102,2019,1,BTR,US,LA,Louisiana,1.0,MQ,1.0,NaN,MLI,BTR-MLI
646798,20191210324,2019,1,BTV,US,VT,Vermont,1.0,AA,1.0,NaN,SWF,BTV-SWF
648348,201913160021,2019,1,CPR,US,WY,Wyoming,1.0,OO,1.0,NaN,EUG,CPR-EUG
648349,201913160021,2019,1,CPR,US,WY,Wyoming,1.0,OO,1.0,NaN,EUG,CPR-EUG
666384,20191433856,2019,1,GEG,US,WA,Washington,1.0,AA,1.0,NaN,EWN,GEG-EWN
675130,201911004520,2019,1,SFO,US,CA,California,1.0,AA,1.0,NaN,ISP,SFO-ISP
675131,201911004520,2019,1,SFO,US,CA,California,1.0,AA,1.0,NaN,ISP,SFO-ISP
675916,20191827519,2019,1,PHL,US,PA,Pennsylvania,1.0,AA,3.0,NaN,SJT,PHL-SJT
682016,201913232309,2019,1,IDA,US,ID,Idaho,1.0,OO,1.0,NaN,EKO,IDA-EKO


In [36]:
tickets = pd.merge(tickets,
                   avg_ticket_price,
                   on = 'TRIP',
                   how = 'left')
tickets.head().T

,0,1,2,3,4
ITIN_ID,201912723049,201912723465,201912723049,201912723465,201913102001
YEAR,2019,2019,2019,2019,2019
QUARTER,1,1,1,1,1
ORIGIN,ABI,ABI,ABI,ABI,ABR
ORIGIN_COUNTRY,US,US,US,US,US
ORIGIN_STATE_ABR,TX,TX,TX,TX,SD
ORIGIN_STATE_NM,Texas,Texas,Texas,Texas,South Dakota
ROUNDTRIP,1.0,1.0,1.0,1.0,1.0
REPORTING_CARRIER,MQ,MQ,MQ,MQ,OO
PASSENGERS,1.0,1.0,1.0,1.0,1.0


In [37]:
tickets['ITIN_FARE_CLEAN'] = np.where(tickets['ITIN_FARE'].isna(), tickets['AVG_FARE'], tickets['ITIN_FARE'])

In [38]:
tickets['ITIN_FARE_CLEAN'].isna().sum()

10

In [39]:
tickets = tickets[~tickets['ITIN_FARE_CLEAN'].isna()]

In [40]:
tickets.head().T

,0,1,2,3,4
ITIN_ID,201912723049,201912723465,201912723049,201912723465,201913102001
YEAR,2019,2019,2019,2019,2019
QUARTER,1,1,1,1,1
ORIGIN,ABI,ABI,ABI,ABI,ABR
ORIGIN_COUNTRY,US,US,US,US,US
ORIGIN_STATE_ABR,TX,TX,TX,TX,SD
ORIGIN_STATE_NM,Texas,Texas,Texas,Texas,South Dakota
ROUNDTRIP,1.0,1.0,1.0,1.0,1.0
REPORTING_CARRIER,MQ,MQ,MQ,MQ,OO
PASSENGERS,1.0,1.0,1.0,1.0,1.0


## Exploring the Flights Data

In [77]:
# From the data dictionary we see that 'FL_DATE' is a date column and hence we parse it into a datetype object 
# while loading the dataset
flights = pd.read_csv('Flights.csv', parse_dates=['FL_DATE'])

/Users/jeet/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#### Size of the data

In [78]:
flights.shape

(1915886, 16)

We have data corresponding to ~1.9 million flights

#### Sample of the data

In [79]:
flights.head().T

,0,1,2,3,4
FL_DATE,2019-03-02 00:00:00,2019-03-02 00:00:00,2019-03-02 00:00:00,2019-03-02 00:00:00,2019-03-02 00:00:00
OP_CARRIER,WN,WN,WN,WN,WN
TAIL_NUM,N955WN,N8686A,N201LV,N413WN,N7832A
OP_CARRIER_FL_NUM,4591,3231,3383,5498,6933
ORIGIN_AIRPORT_ID,14635,14635,14635,14635,14635
ORIGIN,RSW,RSW,RSW,RSW,RSW
ORIGIN_CITY_NAME,"Fort Myers, FL","Fort Myers, FL","Fort Myers, FL","Fort Myers, FL","Fort Myers, FL"
DEST_AIRPORT_ID,11042,11066,11066,11066,11259
DESTINATION,CLE,CMH,CMH,CMH,DAL
DEST_CITY_NAME,"Cleveland, OH","Columbus, OH","Columbus, OH","Columbus, OH","Dallas, TX"


The columns ORIGIN and DESTINATION corresponds to the IATA codes assigned to the aiports. We will use these columns to filter the data for Domestic US based flights

#### Checking the data types

In [80]:
flights.dtypes

FL_DATE              datetime64[ns]
OP_CARRIER                   object
TAIL_NUM                     object
OP_CARRIER_FL_NUM            object
ORIGIN_AIRPORT_ID             int64
ORIGIN                       object
ORIGIN_CITY_NAME             object
DEST_AIRPORT_ID               int64
DESTINATION                  object
DEST_CITY_NAME               object
DEP_DELAY                   float64
ARR_DELAY                   float64
CANCELLED                   float64
AIR_TIME                     object
DISTANCE                     object
OCCUPANCY_RATE              float64
dtype: object

AIR_TIME and DISTANCE needs to be in float/int format. Besides that, the rest of the columns looks appropriate

#### Checking the null count

In [81]:
flights.isna().sum()

FL_DATE                  0
OP_CARRIER               0
TAIL_NUM             12156
OP_CARRIER_FL_NUM        0
ORIGIN_AIRPORT_ID        0
ORIGIN                   0
ORIGIN_CITY_NAME         0
DEST_AIRPORT_ID          0
DESTINATION              0
DEST_CITY_NAME           0
DEP_DELAY            50351
ARR_DELAY            55991
CANCELLED                0
AIR_TIME             56551
DISTANCE               630
OCCUPANCY_RATE         310
dtype: int64

There is no null data in ORIGIN, DESTINATION (IATA CODES) and CANCELLED column. Hence, we do not need to do data imputation before filtering the data for further analysis

#### Checking the distribution of cancelled flights

In [82]:
flights['CANCELLED'].value_counts()

0.0    1864272
1.0      51614
Name: CANCELLED, dtype: int64

Out of ~1.9 million flights, ~51.6K flights have been cancelled. We ignore the cancelled flights and use the remaining set to filter for US based domestic flights

#### Filtering the CANCELLED flights

In [83]:
flights = flights[flights['CANCELLED'] != 1.0]
flights.shape

(1864272, 16)

#### FILTERING the ORIGIN and DESTINATION and MERGING the AIRPORT data

We are interested in domestic US Flights Market. Hence, we will filter the **ORIGIN** and the **DESTINATION** column by US IATA codes we identified in the airport data analysis.

In [84]:
flights = pd.merge(flights,
                   US_airport_subset['IATA_CODE'],
                   left_on = 'ORIGIN',
                   right_on = 'IATA_CODE',
                   how = 'inner')

In [85]:
flights = pd.merge(flights,
                   US_airport_subset['IATA_CODE'],
                   left_on = 'DESTINATION',
                   right_on = 'IATA_CODE',
                   how = 'inner')

In [86]:
flights.drop(['IATA_CODE_x','IATA_CODE_y'], axis = 1, inplace=True)
flights.head().T

,0,1,2,3,4
FL_DATE,2019-03-02 00:00:00,2019-03-09 00:00:00,2019-03-24 00:00:00,2019-03-11 00:00:00,2019-03-10 00:00:00
OP_CARRIER,WN,WN,UA,UA,UA
TAIL_NUM,N955WN,N754SW,N14249,N14240,N11206
OP_CARRIER_FL_NUM,4591,3016,2350,2350,2350
ORIGIN_AIRPORT_ID,14635,14635,14635,14635,14635
ORIGIN,RSW,RSW,RSW,RSW,RSW
ORIGIN_CITY_NAME,"Fort Myers, FL","Fort Myers, FL","Fort Myers, FL","Fort Myers, FL","Fort Myers, FL"
DEST_AIRPORT_ID,11042,11042,11042,11042,11042
DESTINATION,CLE,CLE,CLE,CLE,CLE
DEST_CITY_NAME,"Cleveland, OH","Cleveland, OH","Cleveland, OH","Cleveland, OH","Cleveland, OH"


In [87]:
flights.shape

(1832457, 16)

### Pre-processing and cleaning the flights data

In [88]:
flights['TRIP'] = flights['ORIGIN'] + '-' + flights['DESTINATION']

#### Dtype Correction

In [89]:
flights['DISTANCE'] = flights['DISTANCE'].apply(lambda x: clean_value(x))
flights['AIR_TIME'] = flights['AIR_TIME'].apply(lambda x: clean_value(x))

In [90]:
flights['DISTANCE'] = flights['DISTANCE'].replace('',np.nan).astype('float64')
flights['AIR_TIME'] = flights['AIR_TIME'].replace('',np.nan).astype('float64')

In [91]:
flights.isna().sum()

FL_DATE                 0
OP_CARRIER              0
TAIL_NUM                0
OP_CARRIER_FL_NUM       0
ORIGIN_AIRPORT_ID       0
ORIGIN                  0
ORIGIN_CITY_NAME        0
DEST_AIRPORT_ID         0
DESTINATION             0
DEST_CITY_NAME          0
DEP_DELAY               0
ARR_DELAY            4271
CANCELLED               0
AIR_TIME             6751
DISTANCE             2680
OCCUPANCY_RATE        310
TRIP                    0
dtype: int64

In [92]:
flights.describe()

,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_DELAY,ARR_DELAY,CANCELLED,AIR_TIME,DISTANCE,OCCUPANCY_RATE
count,1.832457e+06,1.832457e+06,1.832457e+06,1.828186e+06,1832457.0,1.825706e+06,1.829777e+06,1.832147e+06
mean,1.268062e+04,1.268198e+04,1.073717e+01,5.599560e+00,0.0,1.089676e+02,7.685074e+02,6.502279e-01
std,1.520450e+03,1.520073e+03,4.984298e+01,5.219427e+01,0.0,7.036551e+01,5.813784e+02,2.020018e-01
min,1.013500e+04,1.013500e+04,-6.300000e+01,-9.400000e+01,0.0,1.000000e+00,2.000000e+00,3.000000e-01
25%,1.129200e+04,1.129200e+04,-6.000000e+00,-1.500000e+01,0.0,5.900000e+01,3.440000e+02,4.800000e-01
50%,1.288900e+04,1.288900e+04,-2.000000e+00,-6.000000e+00,0.0,9.000000e+01,6.060000e+02,6.500000e-01
75%,1.405700e+04,1.405700e+04,7.000000e+00,8.000000e+00,0.0,1.380000e+02,1.005000e+03,8.247386e-01
max,1.621800e+04,1.621800e+04,2.941000e+03,2.923000e+03,0.0,2.222000e+03,9.898000e+03,1.000000e+00


#### Missing value treatment

We take take median of all the numerical values by each trip and then impute the values at the null location

In [93]:
avg_flights_stat = flights.groupby('TRIP')['ARR_DELAY','AIR_TIME','DISTANCE','OCCUPANCY_RATE'].median().reset_index()
avg_flights_stat.head().T

/var/folders/41/3t5xmrtj529186zt9p86v_mh0000gn/T/ipykernel_71548/1883212129.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg_flights_stat = flights.groupby('TRIP')['ARR_DELAY','AIR_TIME','DISTANCE','OCCUPANCY_RATE'].median().reset_index()


,0,1,2,3,4
TRIP,ABE-ATL,ABE-CLT,ABE-DTW,ABE-FLL,ABE-ORD
ARR_DELAY,-6.0,-3.0,-7.5,1.0,-10.0
AIR_TIME,106.0,99.0,83.0,147.0,111.0
DISTANCE,692.0,481.0,425.0,1041.0,654.0
OCCUPANCY_RATE,0.62,0.67,0.63,0.53,0.69


In [94]:
avg_flights_stat.isna().sum()

TRIP              0
ARR_DELAY         5
AIR_TIME          5
DISTANCE          0
OCCUPANCY_RATE    0
dtype: int64

In [95]:
avg_flights_stat['ARR_DELAY'] = avg_flights_stat['ARR_DELAY'].fillna(avg_flights_stat['ARR_DELAY'].mean())
avg_flights_stat['AIR_TIME'] = avg_flights_stat['AIR_TIME'].fillna(avg_flights_stat['AIR_TIME'].mean())

In [96]:
avg_flights_stat.rename(columns={'ARR_DELAY':'MEDIAN_ARR_DELAY',
                                 'AIR_TIME':'MEDIAN_AIR_TIME',
                                 'DISTANCE':'MEDIAN_DISTANCE',
                                 'OCCUPANCY_RATE':'MEDIAN_OCCUPANCY_RATE'},
                        inplace=True)

In [97]:
flights = pd.merge(flights,
                   avg_flights_stat,
                   on = 'TRIP',
                   how = 'inner')

In [98]:
flights['ARR_DELAY'] = np.where(flights['ARR_DELAY'].isna(),
                                flights['MEDIAN_ARR_DELAY'],
                                flights['ARR_DELAY'])

flights['AIR_TIME'] = np.where(flights['AIR_TIME'].isna(),
                               flights['MEDIAN_AIR_TIME'],
                               flights['AIR_TIME'])

flights['DISTANCE'] = np.where(flights['DISTANCE'].isna(),
                               flights['MEDIAN_DISTANCE'],
                               flights['DISTANCE'])

flights['OCCUPANCY_RATE'] = np.where(flights['OCCUPANCY_RATE'].isna(),
                                     flights['MEDIAN_OCCUPANCY_RATE'],
                                     flights['OCCUPANCY_RATE'])

In [99]:
flights.isna().sum()

FL_DATE                  0
OP_CARRIER               0
TAIL_NUM                 0
OP_CARRIER_FL_NUM        0
ORIGIN_AIRPORT_ID        0
ORIGIN                   0
ORIGIN_CITY_NAME         0
DEST_AIRPORT_ID          0
DESTINATION              0
DEST_CITY_NAME           0
DEP_DELAY                0
ARR_DELAY                0
CANCELLED                0
AIR_TIME                 0
DISTANCE                 0
OCCUPANCY_RATE           0
TRIP                     0
MEDIAN_ARR_DELAY         0
MEDIAN_AIR_TIME          0
MEDIAN_DISTANCE          0
MEDIAN_OCCUPANCY_RATE    0
dtype: int64

### Adding base costs to the airport data

In [66]:
US_airport_subset.head()

,TYPE,NAME,ELEVATION_FT,CONTINENT,ISO_COUNTRY,MUNICIPALITY,IATA_CODE,COORDINATES
6194,medium_airport,Aleknagik / New Airport,66.0,NaN,US,Aleknagik,WKK,"-158.617996216, 59.2826004028"
26143,medium_airport,Lehigh Valley International Airport,393.0,NaN,US,Allentown,ABE,"-75.44080352783203, 40.652099609375"
26144,medium_airport,Abilene Regional Airport,1791.0,NaN,US,Abilene,ABI,"-99.68190002440001, 32.4113006592"
26145,large_airport,Albuquerque International Sunport,5355.0,NaN,US,Albuquerque,ABQ,"-106.609001, 35.040199"
26146,medium_airport,Aberdeen Regional Airport,1302.0,NaN,US,Aberdeen,ABR,"-98.42179870605469, 45.449100494384766"


In [67]:
US_airport_subset['COST'] = np.where(US_airport_subset['TYPE'] == 'medium_airport', 5000, 10000)

### Adding late cost and misc cost to the flights data

In [68]:
flights.head().T

,0,1,2,3,4
FL_DATE,2019-03-02 00:00:00,2019-03-09 00:00:00,2019-03-24 00:00:00,2019-03-11 00:00:00,2019-03-10 00:00:00
OP_CARRIER,WN,WN,UA,UA,UA
TAIL_NUM,N955WN,N754SW,N14249,N14240,N11206
OP_CARRIER_FL_NUM,4591,3016,2350,2350,2350
ORIGIN_AIRPORT_ID,14635,14635,14635,14635,14635
ORIGIN,RSW,RSW,RSW,RSW,RSW
ORIGIN_CITY_NAME,"Fort Myers, FL","Fort Myers, FL","Fort Myers, FL","Fort Myers, FL","Fort Myers, FL"
DEST_AIRPORT_ID,11042,11042,11042,11042,11042
DESTINATION,CLE,CLE,CLE,CLE,CLE
DEST_CITY_NAME,"Cleveland, OH","Cleveland, OH","Cleveland, OH","Cleveland, OH","Cleveland, OH"


In [102]:
# $8 per mile (Fuel, Oil, Maintenance, Crew) + $1.18 per mile (Depreciation, Insurance, Other)
flights['MISC_COST'] = flights['DISTANCE'] * 9.18

In [103]:
flights['DEP_LATE_FEES'] = np.where(flights['DEP_DELAY'] > 15, 
                                    75*(flights['DEP_DELAY'] - 15),
                                    0)

flights['ARR_LATE_FEES'] = np.where(flights['ARR_DELAY'] > 15, 
                                    75*(flights['ARR_DELAY'] - 15),
                                    0)

flights['LATE_FEES'] = flights['DEP_LATE_FEES'] + flights['ARR_LATE_FEES']

In [104]:
flights = pd.merge(flights,
                   US_airport_subset[['IATA_CODE','COST']],
                   left_on = 'DESTINATION',
                   right_on = 'IATA_CODE',
                   how = 'inner')
flights.drop(['IATA_CODE'], axis = 1, inplace = True)
flights.rename(columns = {'COST':'DESTINATION_COST'}, inplace = True)

In [105]:
flights[['DEP_LATE_FEES','ARR_LATE_FEES','MISC_COST','OCCUPANCY_RATE']].describe()

,DEP_LATE_FEES,ARR_LATE_FEES,MISC_COST,OCCUPANCY_RATE
count,1.832457e+06,1.832457e+06,1.832457e+06,1.832457e+06
mean,7.757420e+02,7.834344e+02,7.058079e+03,6.502284e-01
std,3.442338e+03,3.435155e+03,5.335966e+03,2.019848e-01
min,0.000000e+00,0.000000e+00,1.836000e+01,3.000000e-01
25%,0.000000e+00,0.000000e+00,3.157920e+03,4.800000e-01
50%,0.000000e+00,0.000000e+00,5.563080e+03,6.500000e-01
75%,0.000000e+00,0.000000e+00,9.253440e+03,8.200000e-01
max,2.194500e+05,2.181000e+05,9.086364e+04,1.000000e+00


In [106]:
flights[flights['DEP_LATE_FEES'] != 0]['DEP_LATE_FEES'].describe()

count    330139.000000
mean       4305.803843
std        7111.456843
min          75.000000
25%         750.000000
50%        2100.000000
75%        5100.000000
max      219450.000000
Name: DEP_LATE_FEES, dtype: float64

In [107]:
flights[flights['ARR_LATE_FEES'] != 0]['ARR_LATE_FEES'].describe()

count    348159.000000
mean       4123.431859
std        6952.413266
min          75.000000
25%         750.000000
50%        1950.000000
75%        4875.000000
max      218100.000000
Name: ARR_LATE_FEES, dtype: float64

In [108]:
flights.head().T

,0,1,2,3,4
FL_DATE,2019-03-02 00:00:00,2019-03-09 00:00:00,2019-03-24 00:00:00,2019-03-11 00:00:00,2019-03-10 00:00:00
OP_CARRIER,WN,WN,UA,UA,UA
TAIL_NUM,N955WN,N754SW,N14249,N14240,N11206
OP_CARRIER_FL_NUM,4591,3016,2350,2350,2350
ORIGIN_AIRPORT_ID,14635,14635,14635,14635,14635
ORIGIN,RSW,RSW,RSW,RSW,RSW
ORIGIN_CITY_NAME,"Fort Myers, FL","Fort Myers, FL","Fort Myers, FL","Fort Myers, FL","Fort Myers, FL"
DEST_AIRPORT_ID,11042,11042,11042,11042,11042
DESTINATION,CLE,CLE,CLE,CLE,CLE
DEST_CITY_NAME,"Cleveland, OH","Cleveland, OH","Cleveland, OH","Cleveland, OH","Cleveland, OH"


#### Aggregating the flight data

In [110]:
flights_agg = flights.groupby(['ORIGIN','DESTINATION','TRIP']).agg({'MISC_COST':'mean',
                                                                    'LATE_FEES':'median',
                                                                    'DESTINATION_COST':'mean',
                                                                    'OCCUPANCY_RATE':'mean'}).reset_index()

In [111]:
flights_agg.describe()

,MISC_COST,LATE_FEES,DESTINATION_COST,OCCUPANCY_RATE
count,5632.000000,5632.000000,5632.000000,5632.000000
mean,8139.268758,42.007724,9161.931818,0.650556
std,5920.374237,781.498208,1867.779981,0.029373
min,284.580000,0.000000,5000.000000,0.300000
25%,3993.300000,0.000000,10000.000000,0.641212
50%,6784.020000,0.000000,10000.000000,0.650342
75%,10382.580000,0.000000,10000.000000,0.660000
max,90863.640000,30225.000000,10000.000000,0.970000


In [112]:
flights_agg.head()

,ORIGIN,DESTINATION,TRIP,MISC_COST,LATE_FEES,DESTINATION_COST,OCCUPANCY_RATE
0,ABE,ATL,ABE-ATL,6352.56,0.0,10000.0,0.641567
1,ABE,CLT,ABE-CLT,4415.58,0.0,10000.0,0.665976
2,ABE,DTW,ABE-DTW,3901.50,0.0,10000.0,0.639073
3,ABE,FLL,ABE-FLL,9556.38,0.0,10000.0,0.580000
4,ABE,ORD,ABE-ORD,6003.72,0.0,10000.0,0.671195


In [113]:
flights_agg['REVERSE_TRIP'] = flights_agg['DESTINATION'] + '-' + flights_agg['ORIGIN']

In [119]:
round_trip_flights = pd.merge(flights_agg,
                              flights_agg,
                              left_on = 'TRIP',
                              right_on = 'REVERSE_TRIP',
                              how = 'inner')

In [121]:
drop_cols = ['ORIGIN_y','DESTINATION_y','TRIP_y','MISC_COST_y','REVERSE_TRIP_y']
round_trip_flights.drop(drop_cols, axis = 1, inplace = True)

In [122]:
round_trip_flights.head().T

,0,1,2,3,4
ORIGIN_x,ABE,ABE,ABE,ABE,ABE
DESTINATION_x,ATL,CLT,DTW,FLL,ORD
TRIP_x,ABE-ATL,ABE-CLT,ABE-DTW,ABE-FLL,ABE-ORD
MISC_COST_x,6352.56,4415.58,3901.5,9556.38,6003.72
LATE_FEES_x,0.0,0.0,0.0,0.0,0.0
DESTINATION_COST_x,10000.0,10000.0,10000.0,10000.0,10000.0
OCCUPANCY_RATE_x,0.641567,0.665976,0.639073,0.58,0.671195
REVERSE_TRIP_x,ATL-ABE,CLT-ABE,DTW-ABE,FLL-ABE,ORD-ABE
LATE_FEES_y,0.0,0.0,0.0,0.0,0.0
DESTINATION_COST_y,5000.0,5000.0,5000.0,5000.0,5000.0


In [125]:
round_trip_flights.rename(columns = {'ORIGIN_x':'ORIGIN',
                                     'DESTINATION_x':'DESTINATION',
                                     'TRIP_x':'TRIP',
                                     'MISC_COST_x':'MISC_COST',
                                     'LATE_FEES_x':'LATE_FEES_ONWARDS',
                                     'DESTINATION_COST_x':'AIRPORT_COST_ONWARD',
                                     'OCCUPANCY_RATE_x':'OCCUPANCY_RATE_ONWARD',
                                     'REVERSE_TRIP_x':'REVERSE_TRIP',
                                     'LATE_FEES_y':'LATE_FEES_RETURN',
                                     'DESTINATION_COST_y':'AIRPORT_COST_RETURN',
                                     'OCCUPANCY_RATE_y':'OCCUPANCY_RATE_RETURN'},
                          inplace = True)

In [126]:
round_trip_flights['LATE_FEES'] = round_trip_flights['LATE_FEES_ONWARDS'] + round_trip_flights['LATE_FEES_RETURN']
round_trip_flights['AIRPORT_COST'] = round_trip_flights['AIRPORT_COST_ONWARD'] + round_trip_flights['AIRPORT_COST_RETURN']

### Calculating the revenue

In [127]:
tickets.head().T

,0,1,2,3,4
ITIN_ID,201912723049,201912723465,201912723049,201912723465,201913102001
YEAR,2019,2019,2019,2019,2019
QUARTER,1,1,1,1,1
ORIGIN,ABI,ABI,ABI,ABI,ABR
ORIGIN_COUNTRY,US,US,US,US,US
ORIGIN_STATE_ABR,TX,TX,TX,TX,SD
ORIGIN_STATE_NM,Texas,Texas,Texas,Texas,South Dakota
ROUNDTRIP,1.0,1.0,1.0,1.0,1.0
REPORTING_CARRIER,MQ,MQ,MQ,MQ,OO
PASSENGERS,1.0,1.0,1.0,1.0,1.0


In [128]:
tickets[['PASSENGERS','ITIN_FARE_CLEAN']].describe()

,PASSENGERS,ITIN_FARE_CLEAN
count,683036.000000,684198.000000
mean,1.938773,472.820514
std,5.090605,340.560906
min,1.000000,0.000000
25%,1.000000,281.000000
50%,1.000000,416.000000
75%,1.000000,596.000000
max,681.000000,38400.000000


In [129]:
tickets_agg = tickets.groupby('TRIP').agg({'ITIN_FARE_CLEAN':'mean'}).reset_index()
tickets_agg.rename(columns={'ITIN_FARE_CLEAN':'FARE_PER_PASSENGER'},inplace = True)

In [130]:
tickets_agg.head()

,TRIP,FARE_PER_PASSENGER
0,ABE-ABQ,534.000000
1,ABE-AGS,299.000000
2,ABE-AMA,654.000000
3,ABE-ASE,742.000000
4,ABE-ATL,505.038462


### Merging the datasets

In [137]:
trip_data = pd.merge(round_trip_flights,
                     tickets_agg,
                     on = 'TRIP',
                     how = 'inner')

In [138]:
trip_data.shape

(5508, 14)

In [140]:
trip_data.head().T

,0,1,2,3,4
ORIGIN,ABE,ABE,ABE,ABE,ABE
DESTINATION,ATL,CLT,DTW,FLL,ORD
TRIP,ABE-ATL,ABE-CLT,ABE-DTW,ABE-FLL,ABE-ORD
MISC_COST,6352.56,4415.58,3901.5,9556.38,6003.72
LATE_FEES_ONWARDS,0.0,0.0,0.0,0.0,0.0
AIRPORT_COST_ONWARD,10000.0,10000.0,10000.0,10000.0,10000.0
OCCUPANCY_RATE_ONWARD,0.641567,0.665976,0.639073,0.58,0.671195
REVERSE_TRIP,ATL-ABE,CLT-ABE,DTW-ABE,FLL-ABE,ORD-ABE
LATE_FEES_RETURN,0.0,0.0,0.0,0.0,0.0
AIRPORT_COST_RETURN,5000.0,5000.0,5000.0,5000.0,5000.0


In [143]:
# Defining revenue metrics
trip_data['NUM_PASSENGERS'] = (trip_data['OCCUPANCY_RATE_ONWARD'] + trip_data['OCCUPANCY_RATE_RETURN'])* 200
trip_data['BASE_REVENUE'] = trip_data['NUM_PASSENGERS'] * trip_data['FARE_PER_PASSENGER']
trip_data['LUGGAGE_REVENUE'] = trip_data['NUM_PASSENGERS'] * 70/2
trip_data['TRIP_REVENUE'] = trip_data['BASE_REVENUE'] + trip_data['LUGGAGE_REVENUE']

In [144]:
# Defining cost metrics
trip_data['MISC_COST_ROUNDTRIP'] = trip_data['MISC_COST']*2
trip_data['TRIP_COST'] = trip_data['AIRPORT_COST'] + trip_data['LATE_FEES'] + trip_data['MISC_COST_ROUNDTRIP']

## Answering the questions